In [1]:
import os, sys, torch

PROJECT_ROOT = '/scratch/jq2uw/derm_vlms'
SKINGPT_DIR = os.path.join(PROJECT_ROOT, 'skingpt')

if SKINGPT_DIR not in sys.path:
    sys.path.insert(0, SKINGPT_DIR)
os.chdir(SKINGPT_DIR)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
torch.cuda.empty_cache()

from model_skingpt4 import init_cfg, init_chat, chat_with_image

print('Loading model...')
cfg = init_cfg(gpu_id=0)
model, vis_processor, chat = init_chat(cfg)
print(f'Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')
print(f'Total params:     {sum(p.numel() for p in model.parameters()):,}')

/home/jq2uw/miniconda3/envs/skingpt4/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...
Initializing Configs
Initializing Chat
Loading VIT


/home/jq2uw/miniconda3/envs/skingpt4/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading VIT Done
Loading Q-Former
Loading Q-Former Done
Loading LLM tokenizer
Loading LLM model


Loading checkpoint shards: 100%|██████████████████████████████████████| 3/3 [00:31<00:00, 10.41s/it]


Loading LLM Done
Load 2 training prompts
Prompt Example 
###Human: <Img><ImageHere></Img> What's wrong with my skin? ###Assistant: 
Load BLIP2-LLM Checkpoint: ./model_skingpt4/weights/skingpt4_llama2_13bchat_base_pretrain_stage2.pth
Initialization Finished
Trainable params: 3,937,280
Total params:     14,110,861,184


In [2]:
import pandas as pd
from PIL import Image
from pathlib import Path

sys.path.insert(0, PROJECT_ROOT)
from data_utils import sample_lesions

DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results')
IMAGES_DIR = os.path.join(RESULTS_DIR, 'images')

df = pd.read_parquet(os.path.join(PROJECT_ROOT, 'data_share', 'midas_share.parquet'))
print(f'Loaded {len(df)} rows')
print(f'y3 distribution:\n{df["y3"].value_counts()}')

SEED = 42
N_PER_CLASS = 5
df_sample = sample_lesions(df, data_dir=DATA_DIR, output_dir=IMAGES_DIR, n_per_class=N_PER_CLASS, seed=SEED)
df_sample.head()

Loaded 3357 rows
y3 distribution:
y3
malignant    1391
benign       1322
other         644
Name: count, dtype: int64
Sampled 10 lesions (5 per class) -> 30 rows
Classes: {'benign': 15, 'malignant': 15}
Images saved to: /scratch/jq2uw/derm_vlms/results/images


,id,ground_truth,image_mode,image_path,original_image_name,lesion_id
0,1_photo,benign,photo,/scratch/jq2uw/derm_vlms/results/images/1_phot...,s-prd-667118134.jpg,534_left lower leg_no
1,1_dscope,benign,dscope,/scratch/jq2uw/derm_vlms/results/images/1_dsco...,s-prd-667118139.jpg,534_left lower leg_no
2,1_combined,benign,combined,/scratch/jq2uw/derm_vlms/results/images/1_comb...,s-prd-667118134.jpg; s-prd-667118139.jpg,534_left lower leg_no
3,2_photo,benign,photo,/scratch/jq2uw/derm_vlms/results/images/2_phot...,s-prd-505211076.jpg,215_central chest _no
4,2_dscope,benign,dscope,/scratch/jq2uw/derm_vlms/results/images/2_dsco...,s-prd-505211307.jpg,215_central chest _no


In [3]:
from tqdm import tqdm

q_describe = "Describe the lesion in detail."
q_classify = "Is the lesion malignant or benign, or other?"
q_describe_classify = q_describe + " " + q_classify
results = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    try:
        image = Image.open(row['image_path']).convert('RGB')
    except Exception as e:
        print(f'[SKIP] {row["id"]}: {e}')
        continue

    results.append({
        'id': row['id'],
        'ground_truth': row['ground_truth'],
        'image_mode': row['image_mode'],
        'describe': chat_with_image(chat, image, q_describe, temperature=0.0, remove_system=True),
        'classify': chat_with_image(chat, image, q_classify, temperature=0.0, remove_system=True),
        'describe_then_classify': chat_with_image(chat, image, q_describe_classify, temperature=0.0, remove_system=True),
        'original_image_name': row['original_image_name'],
        'lesion_id': row['lesion_id'],
    })

print(f'Collected {len(results)} predictions')

  0%|                                                                        | 0/30 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████| 30/30 [07:08<00:00, 14.28s/it]

Collected 30 predictions


In [4]:
results_df = pd.DataFrame(results)
col_order = ['id', 'ground_truth', 'image_mode', 'describe', 'classify',
             'describe_then_classify', 'original_image_name', 'lesion_id']
results_df = results_df[col_order]

out_path = os.path.join(RESULTS_DIR, 'skingpt4_predictions_paired.csv')
results_df.to_csv(out_path, index=False)
print(f'Saved {len(results_df)} rows to {out_path}')

results_df

Saved 30 rows to /scratch/jq2uw/derm_vlms/results/skingpt4_predictions_paired.csv


,id,ground_truth,image_mode,describe,classify,describe_then_classify,original_image_name,lesion_id
0,1_photo,benign,photo,This is an image of a woman's leg and foot. Sh...,This is an image of a woman's leg and foot. Sh...,This is an image of a woman's leg and ankle. S...,s-prd-667118134.jpg,534_left lower leg_no
1,1_dscope,benign,dscope,This image shows a close up view of a person's...,This image appears to be a close up view of a ...,This image appears to be a close up view of a ...,s-prd-667118139.jpg,534_left lower leg_no
2,1_combined,benign,combined,This is an image of a person's foot. The image...,This is an image of a person's arm and leg. Th...,This is an image of a person's ankle and foot....,s-prd-667118134.jpg; s-prd-667118139.jpg,534_left lower leg_no
3,2_photo,benign,photo,The image shows a close up view of the woman's...,This is an image of a woman's neck and chest. ...,The image shows a close up view of a woman's n...,s-prd-505211076.jpg,215_central chest _no
4,2_dscope,benign,dscope,This image shows a close up view of a person's...,This image shows a close up view of a person's...,This is an image of a person's skin with a sma...,s-prd-505211307.jpg,215_central chest _no
5,2_combined,benign,combined,This is an image of a woman's neck and chest. ...,This is an image of a woman's neck and chest. ...,This is an image of a woman's neck and shoulde...,s-prd-505211076.jpg; s-prd-505211307.jpg,215_central chest _no
6,3_photo,benign,photo,"The image shows a small, circular lesion on th...","The image shows a small, dark circle on the pe...","The image shows a small, dark lesion on the ab...",s-prd-529717709.jpg,262_left buttock_no
7,3_dscope,benign,dscope,This image shows a close up view of a lesion o...,This image shows a close up view of a person's...,This image shows a close up view of a lesion o...,s-prd-529718156.jpg,262_left buttock_no
8,3_combined,benign,combined,This image shows a lesion on the thigh of a pe...,This image shows a malignant melanoma on the s...,This is an image of a person's arm. The image ...,s-prd-529717709.jpg; s-prd-529718156.jpg,262_left buttock_no
9,4_photo,benign,photo,The image shows a close up view of the back of...,The image shows a close up view of the back of...,The image shows a close up view of the back of...,s-prd-649658541.jpg,476_mid vertex_no
